# SQLQUeryChain - Mondial - Foreign Key - GPT 3.5

In [1]:

from langchain.sql_database import SQLDatabase

from langchain.chat_models import ChatOpenAI
from langchain.chains import create_sql_query_chain
from urllib.parse import quote  
from langchain.callbacks import get_openai_callback
import time
from dotenv import load_dotenv
import os
import sys
import json
load_dotenv()

experiment_path = '..\..'
path = os.path.abspath('')
module_path = os.path.join(path, experiment_path)
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path+"\\functions")


from sqldatabase_langchain_utils import SQLDatabaseLangchainUtils


<>:14: SyntaxWarning: invalid escape sequence '\.'
<>:14: SyntaxWarning: invalid escape sequence '\.'
C:\Users\Tahsin\AppData\Local\Temp\ipykernel_15540\2926314531.py:14: SyntaxWarning: invalid escape sequence '\.'
  experiment_path = '..\..'


c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\SQLQueryChain\..\..


# Schema

In [2]:
SCHEMA = 'mondial_gpt'
PREFIX = 'mondial'

FILE_NAME_RESULT = f"results/9_sql_queries_chatgpt_{SCHEMA}_fk.json"

In [3]:
def save_queries(queries):
    data = {"queries":queries}
    with open(FILE_NAME_RESULT, "w") as arquivo_json:
        json.dump(data, arquivo_json, indent=4) 

def read_queries():
    with open(FILE_NAME_RESULT, encoding='utf-8', errors='ignore') as json_data:
        data = json.load(json_data, strict=False)
    queries = data["queries"]
    return queries

## Connection

In [4]:
json_file_path = f"../../datasets/{SCHEMA}_db_connection.json"
with open(json_file_path, encoding='utf-8', errors='ignore') as json_data:
    db_connection = json.load(json_data, strict=False)

db_connection

{'DB_HOST': 'localhost',
 'DB_PORT': '1522',
 'DB_USER_NAME': 'MONDIAL_GPT',
 'DB_PASS': 'TextDB123',
 'DB_NAME': '',
 'SQL_DRIVER': 'oracle+oracledb',
 'SERVICE_NAME': 'xepdb1',
 'SCHEMA': 'MONDIAL_GPT',
 'KEYWORD_SEARCH_API_URL': ''}

### Using SQLDatabase to get all the information from the database

In [5]:
db = SQLDatabaseLangchainUtils(db_connection=db_connection)

exclusao = [
    f"{SCHEMA}_tmdp",
    f"{SCHEMA}_tmdpmap",
    f"{SCHEMA}_tmds",
    f"{SCHEMA}_tmjmap",
    f"{SCHEMA}_tpv",
    f"{SCHEMA}_tmdc",
    f"{SCHEMA}_tmdcmap",
    f"{SCHEMA}_tmdej",
    f"{SCHEMA}_log_action",
    f"{SCHEMA}_log_error",
    f"{SCHEMA}_favorite_item", 
    f"{SCHEMA}_favorite_query",
    f"{SCHEMA}_favorite_tag",
    f"{SCHEMA}_favorite_tag_item",
    f"{SCHEMA}_favorite_visualization",
    f"{SCHEMA}_dashboard",
    f"{SCHEMA}_history",
    "teste_cliente",
    "teste_fornecedor",
    "teste_funcionario"
]

include_tables = [s for s in db.get_table_names() if not s.startswith(PREFIX) and s not in exclusao]
db = SQLDatabaseLangchainUtils(db_connection=db_connection, include_tables=include_tables)
db.get_table_names()

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'GEOCOORD' of column 'coordinates'
  self._metadata.reflect(
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'GEOCOORD' of column 'source'
  self._metadata.reflect(
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'GEOCOORD' of column 'estuary'
  self._metadata.reflect(


['airport',
 'borders',
 'city',
 'citylocalname',
 'cityothername',
 'citypops',
 'continent',
 'country',
 'countrylocalname',
 'countryothername',
 'countrypops',
 'desert',
 'economy',
 'encompasses',
 'ethnicgroup',
 'geo_desert',
 'geo_estuary',
 'geo_island',
 'geo_lake',
 'geo_mountain',
 'geo_river',
 'geo_sea',
 'geo_source',
 'island',
 'islandin',
 'ismember',
 'lake',
 'lakeonisland',
 'language',
 'located',
 'locatedon',
 'mergeswith',
 'mountain',
 'mountainonisland',
 'organization',
 'politics',
 'population',
 'province',
 'provincelocalname',
 'provinceothername',
 'provpops',
 'religion',
 'river',
 'riveronisland',
 'riverthrough',
 'sea']

In [6]:
len(include_tables)

46

## Creating the prompt

In [7]:
from langchain.prompts.prompt import PromptTemplate

f = open(f"prompts/prompt_template_sql_query_chain.txt", "r")
prompt_template = f.read()
f.close()


PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "top_k"], template=prompt_template
)

print(PROMPT)

input_variables=['input', 'table_info', 'top_k'] input_types={} partial_variables={} template='You are an Oracle SQL expert. Given an input question, first create a syntactically correct Oracle SQL query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, don\'t query for at {top_k} most results or any using the FETCH FIRST n ROWS ONLY clause as per Oracle SQL. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question.\nPay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\nPay attention to use TRUNC(SYSDATE) function to get the current date, if the question involves "today". \n\nSome hints:\n- Don\'t u

## Creating Chain to generate SQL

In [8]:
query_chain  = create_sql_query_chain(ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo-16k'), db.db, prompt=PROMPT)


C:\Users\Tahsin\AppData\Local\Temp\ipykernel_15540\771061558.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  query_chain  = create_sql_query_chain(ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo-16k'), db.db, prompt=PROMPT)


### Test: query and Number of Tokens

In [9]:
with get_openai_callback() as cb:
    sql_query = query_chain.invoke({"question":"What are the provinces with an area more than 10000?"})
    
    print(cb.total_tokens)
    print(cb.prompt_tokens)
    print(cb.completion_tokens)
    print(cb.total_cost)
sql_query

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:317: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:329: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables


5970
5958
12
0.017922


'SELECT name\nFROM province\nWHERE area > 10000'

In [10]:
with get_openai_callback() as cb:
    sql_query = query_chain.invoke({"question":"What are the languages spoken in Poland?"})
    print(cb.total_tokens)
    print(cb.prompt_tokens)
    print(cb.completion_tokens)
    print(cb.total_cost)
sql_query

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:317: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:329: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "lake, river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables


5963
5953
10
0.017898999999999998


"SELECT name FROM language WHERE country = 'POL'"

## Preparing natural language queries to run in LLM

In [11]:

json_file_path = f"../../datasets/{PREFIX}/queries_{PREFIX}.json"
with open(json_file_path, encoding='utf-8', errors='ignore') as json_data:
    queries = json.load(json_data, strict=False)
queries = queries['queries']
queries

[{'id': '1',
  'question': 'What is the area of Thailand?',
  'query_string': '',
  'type': 'simple'},
 {'id': '2',
  'question': 'What are the provinces with an area greater than 10000?',
  'query_string': '',
  'type': 'simple'},
 {'id': '3',
  'question': 'What are the languages spoken in Poland?',
  'query_string': '',
  'type': 'medium'},
 {'id': '4',
  'question': 'How deep is Lake Kariba?',
  'query_string': '',
  'type': 'simple'},
 {'id': '5',
  'question': 'What is the total of provinces of Netherlands?',
  'query_string': '',
  'type': 'complex'},
 {'id': '6',
  'question': 'What is the percentage of religious people are hindu in thailand?',
  'query_string': '',
  'type': 'complex'},
 {'id': '7',
  'question': 'List the number of provinces each river flows through.',
  'query_string': '',
  'type': 'medium'},
 {'id': '8',
  'question': 'Find all countries that became independent between 8/1/1910 and 8/1/1950.',
  'query_string': '',
  'type': 'complex'},
 {'id': '9',
  'que

# Running queries in LLM to generate SQL

In [14]:

# A cada X consultas, vai ser gerado um delay de 10s para evitar o bloqueio da API.
import warnings
from sqlalchemy import exc

# Suppress SAWarning
warnings.filterwarnings("ignore", category=exc.SAWarning, message=".*Cannot correctly sort tables.*")

# Your code here



number_of_queries_to_delay = 25
count = 0
for instance in queries:
    if count == number_of_queries_to_delay:
        count = 0
        time.sleep(10)
    with get_openai_callback() as cb:
        start_time = time.time()
        sql_query = query_chain.invoke({"question":instance["question"]})
        end_time = time.time()
        instance["query_string"] = sql_query
        instance['total_tokens'] = cb.total_tokens
        instance['prompt_tokens'] = cb.prompt_tokens
        instance['completion_tokens'] = cb.completion_tokens
        instance['total_cost'] = cb.total_cost
        instance['time'] = end_time - start_time
        print(instance['id'], instance['question'], instance["query_string"], instance['time'], instance['total_cost'])
    save_queries(queries)
    count += 1
queries

1 What is the area of Thailand? SELECT area FROM country WHERE name = 'Thailand' 0.6244115829467773 0.0179
2 What are the provinces with an area greater than 10000? SELECT name
FROM province
WHERE area > 10000 0.6955037117004395 0.017922
3 What are the languages spoken in Poland? SELECT name FROM language WHERE country = 'POL' 0.7199192047119141 0.017898999999999998
4 How deep is Lake Kariba? SELECT depth
FROM lake
WHERE name = 'Lake Kariba' 0.5390102863311768 0.017912
5 What is the total of provinces of Netherlands? SELECT COUNT(*) FROM province WHERE country = 'NLD' 0.6090466976165771 0.01791
6 What is the percentage of religious people are hindu in thailand? SELECT percentage
FROM religion
WHERE country = 'THA' AND name = 'Hindu' 1.0385754108428955 0.017961
7 List the number of provinces each river flows through. SELECT r.name, COUNT(DISTINCT rt.province) AS num_provinces
FROM river r
JOIN riverthrough rt ON r.name = rt.river
GROUP BY r.name
ORDER BY num_provinces DESC; 0.9452486038

[{'id': '1',
  'question': 'What is the area of Thailand?',
  'query_string': "SELECT area FROM country WHERE name = 'Thailand'",
  'type': 'simple',
  'total_tokens': 5963,
  'prompt_tokens': 5952,
  'completion_tokens': 11,
  'total_cost': 0.0179,
  'time': 0.6244115829467773},
 {'id': '2',
  'question': 'What are the provinces with an area greater than 10000?',
  'query_string': 'SELECT name\nFROM province\nWHERE area > 10000',
  'type': 'simple',
  'total_tokens': 5970,
  'prompt_tokens': 5958,
  'completion_tokens': 12,
  'total_cost': 0.017922,
  'time': 0.6955037117004395},
 {'id': '3',
  'question': 'What are the languages spoken in Poland?',
  'query_string': "SELECT name FROM language WHERE country = 'POL'",
  'type': 'medium',
  'total_tokens': 5963,
  'prompt_tokens': 5953,
  'completion_tokens': 10,
  'total_cost': 0.017898999999999998,
  'time': 0.7199192047119141},
 {'id': '4',
  'question': 'How deep is Lake Kariba?',
  'query_string': "SELECT depth\nFROM lake\nWHERE na

In [15]:
queries=read_queries()
queries


[{'id': '1',
  'question': 'What is the area of Thailand?',
  'query_string': "SELECT area FROM country WHERE name = 'Thailand'",
  'type': 'simple',
  'total_tokens': 5963,
  'prompt_tokens': 5952,
  'completion_tokens': 11,
  'total_cost': 0.0179,
  'time': 0.6244115829467773},
 {'id': '2',
  'question': 'What are the provinces with an area greater than 10000?',
  'query_string': 'SELECT name\nFROM province\nWHERE area > 10000',
  'type': 'simple',
  'total_tokens': 5970,
  'prompt_tokens': 5958,
  'completion_tokens': 12,
  'total_cost': 0.017922,
  'time': 0.6955037117004395},
 {'id': '3',
  'question': 'What are the languages spoken in Poland?',
  'query_string': "SELECT name FROM language WHERE country = 'POL'",
  'type': 'medium',
  'total_tokens': 5963,
  'prompt_tokens': 5953,
  'completion_tokens': 10,
  'total_cost': 0.017898999999999998,
  'time': 0.7199192047119141},
 {'id': '4',
  'question': 'How deep is Lake Kariba?',
  'query_string': "SELECT depth\nFROM lake\nWHERE na

## Prompt Generated by Langchain

In [24]:
sql_query_chain_prompt = query_chain.middle[1].template.format(table_info=db.get_table_info(), top_k=0, input="{input}")
print(sql_query_chain_prompt)

You are an Oracle SQL expert. Given an input question, first create a syntactically correct Oracle SQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, don't query for at 0 most results or any using the FETCH FIRST n ROWS ONLY clause as per Oracle SQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use TRUNC(SYSDATE) function to get the current date, if the question involves "today". 

Some hints:
- Don't use double quotes in column name
- 
Example:
`SELECT "column_name" FROM table` should be `SELECT column_name FR

In [25]:

f = open(f"prompt_generated_by_langchain_{SCHEMA}_fk.txt", "w")
f.write(sql_query_chain_prompt)
f.close()


#### Fixing queries

In [26]:
to_fix = [40,59,62,72,85,99]
for pos in to_fix:
    instance = queries[pos]
    q = read_queries()
    with get_openai_callback() as cb:
            start_time = time.time()
            sql_query = query_chain.invoke({"question":instance["question"]})
            end_time = time.time()
            instance["query_string"] = sql_query
            instance['total_tokens'] = cb.total_tokens
            instance['prompt_tokens'] = cb.prompt_tokens
            instance['completion_tokens'] = cb.completion_tokens
            instance['total_cost'] = cb.total_cost
            instance['time'] = end_time - start_time
            q[pos] = instance
            print(instance['id'], instance['question'], instance["query_string"], instance['time'], instance['total_cost'])
            save_queries(q)

41 How many countries that are close to the Mediterranean Sea? SELECT COUNT(*) 
FROM encompasses 
WHERE continent = 'Europe' AND percentage >= 10 1.0444142818450928 0.01794
60  List the names of capital cities which are the base for organizations in alphabetical order SELECT city.name
FROM city
JOIN organization ON city.name = organization.city
ORDER BY city.name ASC 0.5610127449035645 0.01797
63 Show the inflation rate of countries that are washed by the Arabian Sea SELECT c.name, e.inflation
FROM country c
JOIN geo_sea gs ON c.code = gs.country
JOIN sea s ON gs.sea = s.name
JOIN economy e ON c.code = e.country
WHERE s.name = 'Arabian Sea' 1.8446147441864014 0.018097000000000002
73 What area is the largest continent? SELECT name, area
FROM continent
ORDER BY area DESC 1.307746410369873 0.017904
86 What are the 3 airports with the largest name? SELECT name
FROM airport
ORDER BY LENGTH(name) DESC
FETCH FIRST 3 ROWS ONLY 1.388308048248291 0.017948000000000002
100 What is the percentage o